# Manipulation de ressources: Fonctionnement détaillé

Initialisation de vortex...

In [1]:
%load_ext ivortex
%vortex tmpcocoon
from bronx.stdtypes import date as bdate

# [2019/01/24-14:03:21][vortex.sessions][_set_rundir:0152][WARNING]: Session <root> is changing its working directory </sauvegarde/stage-vortex/presentation/nb>
# [2019/01/24-14:03:21][vortex.sessions][_set_rundir:0155][INFO]: Session <root> set rundir </home/meunierlf/vortex-workdir/auto_cocoon_rd2b17h3>


Vortex 1.6.0 loaded ( Thursday 24. January 2019, at 14:03:19 )
The working directory is now: /home/meunierlf/vortex-workdir/auto_cocoon_rd2b17h3/root


## Exemple précédemment utilisé...

In [2]:
# Préréglages
toolbox.defaults(
    date = vortex.tools.date.Date('2017060118'),
    cutoff = 'production',
    model = 'arpege',
    geometry = vortex.data.geometries.get(tag='euroc25'),
)
t.glove.vapp = 'arpege'
t.glove.vconf = 'pearp'  # En l'état de nos connaissances, je ne peux expliquer cela...
# Toolbox
toolbox.active_now = True
toolbox.active_verbose = False

In [3]:
# Récupération de la ressource
rhandlers = toolbox.input(# Method/Section
                          loglevel='warning', role='InitialCondition', intent='inout',
                          # Resource
                          term=3, kind='gridpoint', origin='historic', nativefmt='grib',
                          # Provider
                          member=2, experiment='DBLE', namespace='vortex.archive.fr',
                          block='forecast',
                          # Container
                          local='GRIB_[geometry:area]_[member%04d]+[term:fmthm]', format='grib')
rh = rhandlers[0]  # Petit raccourci pour la suite
%ls
rh.clear()

GRIB_EUROC25_0002+0003:00


True

## Récupération d'une ressource "à la main"

Comme vu précédemment, la méthode **get** du resource's *Handler* se charge de la récupération des données. Dans la suite de ce document, dans un but pédagogique, nous allons reproduire manuellement les différentes étapes qui aboutissent à la récupération d'une donnée.

### 1. Génération d'une URI à partir des objets Resource et Provider...

In [4]:
uri = rh.location()  # Plus loin dans la présentation, nous détaillerons cela.
print(uri)
p_uri = vortex.tools.net.uriparse(uri)
print('  Scheme: {}\n  Netloc: {}\n  Path: {}\n  Query: {}\n'.
      format(p_uri['scheme'], p_uri['netloc'], p_uri['path'], p_uri['query']))

vortex://vsop.archive.fr/arpege/pearp/DBLE/20170601T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0003:00.grib
  Scheme: vortex
  Netloc: vsop.archive.fr
  Path: /arpege/pearp/DBLE/20170601T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0003:00.grib
  Query: {}



L'URI contient toutes les informations nécessaires à la récupération des données. C'est le rôle des objets *Store* de récupérer effectivement ces données à partir des informations contenues dans l'URI.

### 2. Création de l'objet Store à même de gérer cette demande...

In [5]:
rh_store = fp.proxy.store(scheme=p_uri['scheme'], netloc=p_uri['netloc'])
    # Note: si elles sont présentes dans l'objet Handler, toutes les options commencant par
    # "stor" sont passées à **footprints** lors de la création du store.
print(rh_store.scheme)
print(rh_store.netloc)

vortex
vsop.archive.fr


Les stores héritant de *ArchiveStore* ont quelques options intéressantes (pour les autres Stores ces options seront ignorées) :

* *storage* : Nom de la machine cible pour l'archivage (``hendrix.meteo.fr`` par défault)
* *storehash* : Lors de l'archivage, création d'une somme de contôle (en utilisant l'algorithme précisé en argument : voir **vortex.util.hash**). Lors de la récupération d'une ressource, vérification de la somme.
* *store_compressed* : Lors de l'archivage, compression des données à la volée (en utilisant l'algorithme précisé en argument : voir **vortex.tools.compression**). Lors de la récupération d'une ressource, décompression à la volée.

### 3. Récupération des données à proprement parler

In [6]:
print("URI's path:", p_uri['path'])
print("Physical location:", rh_store.locate(p_uri))
print("Local destination:", rh.container.iotarget())

URI's path: /arpege/pearp/DBLE/20170601T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0003:00.grib
Physical location: meunierlf@hendrix.meteo.fr:/home/m/mxpt/mxpt001/vortex/arpege/pearp/DBLE/2017/06/01/T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0003:00.grib
Local destination: GRIB_EUROC25_0002+0003:00


In [7]:
rh_store.get(p_uri, rh.container.iotarget(), options=dict(intent='inout', fmt='grib'))
%ls

# [2019/01/24-14:03:22][vortex.data.stores][inarchiveget:1199][INFO]: inarchiveget on vortex://vsop.archive.fr//home/m/mxpt/mxpt001/vortex/arpege/pearp/DBLE/2017/06/01/T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0003:00.grib (to: GRIB_EUROC25_0002+0003:00)
# [2019/01/24-14:03:22][vortex.tools.storage][_ftpretrieve:0690][INFO]: ftpget on ftp://hendrix.meteo.fr//home/m/mxpt/mxpt001/vortex/arpege/pearp/DBLE/2017/06/01/T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0003:00.grib (to: GRIB_EUROC25_0002+0003:00)
# [2019/01/24-14:03:22][vortex.tools.net][get:0266][INFO]: FTP <get:/home/m/mxpt/mxpt001/vortex/arpege/pearp/DBLE/2017/06/01/T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0003:00.grib>


GRIB_EUROC25_0002+0003:00


In [8]:
rh.clear()  # Nettoyage pour la suite de la démo...

True

### Résumé

Les étapes vues précédemment, sont cachées dans la méthode **get** du resource's *Handler* :

* En pratique, la détermination de l'URI et la création du *Store* sont déléguées à la méthode 
  **location** et à la *property* **store**  du resource's *Handler*.
* L'appel à la méthode **get** du *Store* (avec les différentes options nécessaires) se fait au sein de la méthode **get** du resource's *Handler*.
* En réalité, le code de la méthode **get** du resource's *Handler* fait un peu plus de choses 
  notamment du fait de fonctionnalités que nous n'avons pas encore abordées (*insitu*, *hooks*, 
  vérification des métadonnées, *Contents*, ...) ; néanmoins, le plus important est ce qui a été évoqué dans les diapos précédentes...

Tout cela est transposable aux autres méthodes du resource's *Handler* que sont **check** (vérification de la présence d'une donnée dans le *Store*), **locate** (emplacement physique des données), **put** (envoi d'une donnée vers le *Store*) et **delete** (effacement d'une donnée dans le *Store*). Dans tous les cas, on génère l'URI, on crée le *Store*, on appelle la méthode du *Store* qui correspond.

### Quelques remarques sur le rôle des différents objets

* *Resource* + *Provider* : génération de l'URI qui est caractéristique de ce couple. 
  Idéalement, l'URI doit contenir des informations sur l'ensemble des attributs des objets
  *Resource* et *Provider* afin d'éviter toute confusion.
* resource's *Handler* : Rôle de facilitateur uniquement. Il n'influence pas le contenu de 
  l'URI ni l'endroit où la donnée sera physiquement stockée.
* *Store* : Traduction de l'URI en emplacement physique réel. Les actions réalisées par le 
  *Store* ne dépendent que de l'URI (en particulier, on ne s'amuse pas à inspecter le nom
  du fichier pour retrouver quelle est la ressource correspondante).

## Détails sur la génération de l'URI

L'URI est générée par la méthode **location** du resource's *Handler*. Cette méthode délègue en fait le travail à la méthode **uri** du *Provider* à laquelle on passe la *Resource* en argument:

In [9]:
print(rh.location())
print(rh.provider.uri(rh.resource))

vortex://vsop.archive.fr/arpege/pearp/DBLE/20170601T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0003:00.grib
vortex://vsop.archive.fr/arpege/pearp/DBLE/20170601T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0003:00.grib


C'est donc bien le *Provider* qui fait le travail avec l'aide de la *Resource*. En pratique, la méthode **uri** ne fait pas grand chose sinon appeler d'autres méthodes qui se chargeront de construire l'URI morceau par morceau :

In [10]:
print('  Scheme: {}\n  Netloc: {}\n  Path: {}\n  Query: {}\n'.
      format(rh.provider.scheme(rh.resource), rh.provider.netloc(rh.resource),
             rh.provider.pathname(rh.resource) + '/' + rh.provider.basename(rh.resource),
             rh.provider.urlquery(rh.resource)))

  Scheme: vortex
  Netloc: vsop.archive.fr
  Path: arpege/pearp/DBLE/20170601T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0003:00.grib
  Query: None



C'est globalement là que s'arrête la description générale : l'implémentation de ces différentes routines va en effet varier d'un type de *Provider* à un autre. Voici néanmoins quelques indications sur la façon dont ces morceaux d'URI sont construits.

### *scheme* & *netloc*

In [11]:
print(rh.provider.scheme(rh.resource))
print(rh.provider.netloc(rh.resource))

vortex
vsop.archive.fr


*scheme* et *netloc* sont, en général, déterminés directement par le *Provider* (sans recourir à la *Resource*).

### *pathname* et *basename*

In [12]:
print(rh.provider.pathname(rh.resource))
print(rh.provider.basename(rh.resource))

arpege/pearp/DBLE/20170601T1800P/mb002/forecast
grid.arpege-forecast.euroc25+0003:00.grib


On remarque que le résultat dépend beaucoup de la *Resource* à laquelle on s'intéresse. Dès lors, plusieurs approches permettent d'arriver au résultat...

#### 1. Gérer intégralement les choses au niveau du *Provider*

Le *Provider* inspecte les attributs de la *Resource* passée en argument et détermine un chemin/un nom. C'est l'approche suivie par le *Provider* oper dit "inline" (i.e. les données de la chaîne ksh résidant sur le calculateur dans /chaine) :

In [13]:
import iga
iga_provider = fpx.provider(suite='oper', namespace='oper.inline.fr')
print('path={:s} name={:s}'
      .format(iga_provider.pathname(rh.resource), iga_provider.basename(rh.resource)))

path=arpege/pearp/oper/data/bdap name=PErDH003EUROC25


* Aller voir l'implémentation des méthodes **basename** et **pathname** de **iga.data.providers.IgaProvider**.
* On remarque que l'essentiel du travail est délégué aux fonctions **global_bnames** et **global_pnames** du module **iga.util.bpnames**.
* Aller voir l'implémentation de ces méthodes. La *Ressource* est passée au crible (succession de tests *if*) ; il en ressort magiquement un chemin et un nom de fichier.

#### 2. Déléguer tout ou partie du travail à la *Resource* et aller à la pêche au informations

* Pour le *basename*, on appelle la méthode **basename** sur la *Resource* passée en argument. Cette méthode renvoie un nom de ressource ou une ébauche de nom. Dans le cas où il s'agit d'une ébauche de nom, c'est le provider qui finit le travail : cette solution n'est pas idéale mais est malheureusement nécessaire pour supporter des archives "historiques".
* Pour le *pathname*, on appelle la méthode **pathinfo** sur la *Resource* passée en argument. Cette méthode renvoie un dictionnaire contenant les informations nécessaires à la construction du chemin par le provider (souvent la date et le cutoff).

C'est l'approche suivie par le *Provider* Olive :

In [14]:
import olive
olive_provider = fpx.provider(experiment='ABCD', namespace='olive.archive.fr', block='forecast')
print('path={:s} name={:s}'
      .format(olive_provider.pathname(rh.resource), olive_provider.basename(rh.resource)))

path=ABCD/20170601H18P/forecast name=GRIDHSTEUROC25+0003


* Aller voir l'implémentation de la méthode **basename** de **olive.data.providers.Olive**.
* On remarque que tout le travail est délégué à la *Resource* !

Creusons un peu plus ce qui se cache derrière l'arbre d'appel de **basename**... Si l'on suit l'héritage, `super(Olive, self).basename(resource)` renvoie au code suivant : `resource.basename(self.realkind)`. Or, la méthode **basename** de la classe mère (**vortex.data.resources.Resource**) se charge de renvoyer vers la méthode nommée `"{}_basename'.format(self.realkind)"` (ou, à défaut, `"generic_basename"`). Dans ce cas précis, on arrive sur la méthode **olive_basename**.

In [15]:
print("** Provider's realkind:", olive_provider.realkind)
print("** Basename via *basename*:", rh.resource.basename(olive_provider.realkind))
print("** Basename via *olive_basename*:", rh.resource.olive_basename())

** Provider's realkind: olive
** Basename via *basename*: GRIDHSTEUROC25+0003
** Basename via *olive_basename*: GRIDHSTEUROC25+0003


* Aller voir l'implémentation de la méthode **pathname** de **olive.data.providers.Olive**.

Creusons un peu plus ce qui se cache derrière l'arbre d'appel de **pathinfo**... Si l'on suit l'héritage, `super(Olive, self).pathinfo(resource)` renvoie au code suivant : `resource.pathinfo(self.realkind)`. Or, la méthode **pathinfo** de la classe mère **vortex.data.resources.Resource** se charge de renvoyer vers la méthode nommée `"{}_pathinfo'.format(self.realkind)"` (ou, à défaut, `"generic_pathinfo"`). Dans ce cas précis, on arrive sur la méthode **generic_pathinfo**.

In [16]:
print("** Provider's realkind:", olive_provider.realkind)
print("** Basename via *basename*:\n", rh.resource.pathinfo(olive_provider.realkind))
print("** Basename via *olive_basename*:\n", rh.resource.generic_pathinfo())

** Provider's realkind: olive
** Basename via *basename*:
 {'geometry': <vortex.data.geometries.LonlatGeometry object at 0x7efd041e3ba8>, 'nativefmt': 'grib', 'model': 'arpege', 'cutoff': 'production', 'date': Date(2017, 6, 1, 18, 0)}
** Basename via *olive_basename*:
 {'geometry': <vortex.data.geometries.LonlatGeometry object at 0x7efd041e3ba8>, 'nativefmt': 'grib', 'model': 'arpege', 'cutoff': 'production', 'date': Date(2017, 6, 1, 18, 0)}


#### 3. Aller à la pêche au informations et déléguer le travail à une "Fabrique de noms"

On appelle la méthode **namebuilding_info** sur la *Resource* passée en argument. Cette méthode renvoie un dictionnaire contenant les informations nécessaires à la construction du chemin et du nom. Le *Provider* utilise ensuite un objet dédié (la fabrique de noms) afin de générer le chemin et le nom final.

In [17]:
print(rh.provider.pathname(rh.resource))
print(rh.provider.basename(rh.resource))

arpege/pearp/DBLE/20170601T1800P/mb002/forecast
grid.arpege-forecast.euroc25+0003:00.grib


In [18]:
binfos = rh.resource.namebuilding_info()  # On commence par récupérer les infos sur la ressource
binfos

{'flow': [{'date': Date(2017, 6, 1, 18, 0)}, {'shortcutoff': 'production'}],
 'fmt': 'grib',
 'geo': 'EUROC25',
 'radical': 'grid',
 'src': ['arpege', 'forecast'],
 'term': '0003:00'}

À partir de ce dictionnaire, le *provider* Vortex (ses méthodes **basename** et **pathname**) va utiliser un objet de classe **vortex.tools.names.VortexNameBuilder** pour trouver le chemin / le nom de la ressource :

In [19]:
nbuilder = vortex.tools.names.VortexNameBuilder()
nbuilder.pack_basename(binfos)

'grid.arpege-forecast.euroc25+0003:00.grib'

In [20]:
binfos_plus = binfos.copy()
binfos_plus.update(dict(vapp=rh.provider.vapp, vconf=rh.provider.vconf,
                        experiment=rh.provider.experiment, block=rh.provider.block,
                        member=rh.provider.member, scenario=rh.provider.scenario,))
nbuilder.pack_pathname(binfos_plus)

'arpege/pearp/DBLE/20170601T1800P/mb002/forecast'

Les noms Vortex sont donc auto-générés, ce qui leur confère une certaine cohérence.

### *urlquery*

Cette portion de l'URI n'est pas très couramment utilisée. Néanmoins, elle peut servir à donner des indications supplémentaires au *Store*.

In [21]:
print(rh.provider.urlquery(rh.resource))
# Ok ce cas n'est pas très spectaculaire...

None


Les providers recourant à ce mécanisme, appellent la méthode **urlquery** sur la *Resource*. Là aussi on scrute la *Resource* afin de déterminer s'il existe une méthode dédiée pour tel ou tel *Provider* (par défaut, **vortex_urlquery** est utilisée).

### Récapitulatif

On voit désormais un peu plus ce qui se passe lors de l'appel à :

In [22]:
print(rh.location())

vortex://vsop.archive.fr/arpege/pearp/DBLE/20170601T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0003:00.grib


La conséquence pratique de cela est que, lors de la création d'un nouvel objet *Resource*, il faut surcharger ou créer les méthodes voulues pour permettre aux *providers* de créer une URI.

La notion de surcharge est importante car il est souvent possible de récupérer beaucoup des méthodes de la classe dont on hérite (en particulier les méthodes **namebuilding_info** et **generic_pathinfo** qui sont rarement ré-implémentées).

Dans le cas du provider Vortex, la fabrique de noms joue un rôle central : on donne quelques détails dans le paragraphe suivant.

### Fabrique de noms Vortex

La fabrique de noms est utilisée à la fois pour la génération du *pathname* et du *basename* :

* Le cas du *pathname* ne devrait pas trop préoccuper les dévelopeurs car le comportement est normalement déterminé par le classe dont on hérite. Par exemple, si une ressource hérite de la classe **FlowResource** (voir présentation précédente), celle-ci recevra en "héritage" des attributs *date* et *cutoff* qui apparaîtront dans le dictionnaire retourné par **namebuilding_info**.
* Pour partie, le même résonnement peut être tenu pour le *basename*. Par exemple, en héritant de **GeoFlowResource**, les attributs *model* et *geometry* seront hérités et utilisés dans la génération de noms. En revanche, pour des attributs qui seraient ajoutés "à la main" et/ou orginaux (dans le sens où ils seraient spécifiques à tel ou tel usage), c'est au développeur de faire en sorte qu'ils soient transmis à la fabrique de noms.

Au vu de l'explication précédente, on recourera donc massivement :

* A l'héritage de classes (utilisation de classes abstraites comme **FlowResource**, **GeoFlowResource**, ...) ;
* Pour les attributs "classiques", on se reposera sur leur pré-définition. Par exemple, pour ajouter une échéance (attribut *term*), on utilisera : **vortex.syntax.stdattrs.term_deco** (voir aussi http://intra.cnrm.meteo.fr/algopy/sphinx/vortex/current/library/vortex/syntax/stdattrs.html) ;
* Pour les attributs "originaux", le développeur ajoutera les informations nécessaires dans le dictionnaire retourné par la méthode **namebuilding_info**.

Concrétement, le contenu du dictionnaire renvoyé par **namebuilding_info** doit répondre à certaines exigences. Voici une version maximaliste de ce qui pourrait être contenu dans celui-ci (bien sûr, selon le contexte, certains éléments seront omis) :

In [23]:
bdict = dict(# Un préfixe. Par défaut: valeur retournée par self.realkind -> ne pas modifier
             radical='ressourceid',  
             # Des informations indiquant la provenance/la nature des données. Par défaut, si un
             # attribut *model* est défini, il figure ici -> ajouter à cette liste des éléments
             # qui vous seraient propres.
             src=['arpege', 'somethingelse'],  
             # Si les données ont subi un filtrage, l'indiqué ici
             filtername = 'filtre',
             # Des informations sur la géométrie utilisée -> ne pas modifier (le défaut est
             # fourni par l'héritage ou vortex.data.geometries.hgeometry_deco)
             geo = [dict(trunctation=1198), dict(stretching=2.4)],
             # Des informations sur la parallélisation
             compute = [dict(mpi=1), dict(openmp=1)],
             # L'échéance de la ressource -> ne pas modifier (vortex.syntax.stdattrs.term_deco
             # s'occupe de tout)
             term = bdate.Time('24:00').fmthm,
             # Un numéro -> ne pas modifier (vortex.syntax.stdattrs.number_deco s'occupe de tout)
             number = vortex.syntax.stdattrs.FmtInt(1, fmt='03'),
             # Le format des données -> ne pas modifier (vortex.syntax.stdattrs.nativefmt_deco
             # s'occupe de tout)
             fmt = 'grib',
             # Un Suffixe. Peu utilisé. (par exemple, vortex.syntax.stdattrs.month_deco,
             # l'utilise pour le numero du mois)
             suffix = 'm12'
            )
nbuilder.pack_basename(bdict)

'ressourceid.arpege-somethingelse.filtre.1198-c24.n0001-omp01+0024:00.001.grib.m12'

Nous verrons dans la présentation suivante un cas pratique de création d'une ressource. Pour faire simple :

* Si vous avez besoin d'un attribut de façon récurrente, ne pas hésiter à le pré-définir (en créant une *footprint* abstraite pour celui-ci. Cela peut se faire dans **vortex.syntax.stdattrs** ou dans votre propre package (*e.g.* **cen.syntax.stdattrs**)
* Pour les autres attributs originaux, surcharger **realkind** et **namebuilding_info**. (*NB*: la surcharge de **namebuilding_info** peut se faire "à la main" ou via les décorateurs de classe définis dans **vortex.syntax.stddeco** : un exemple sera donné dans la présentation suivante)

Nous avons, jusque là, parlé de "**LA** fabrique de noms", à vrai dire nous devrions plutôt dire "la fabrique de noms standard". En effet, il est théoriquement possible de créer une fabrique de noms personnalisée et de l'utiliser dans le provider *Vortex* (via son attribut *namebuild* : "la fabrique de noms standard" correspond en fait à *namebuild = 'std'*). Cette pratique est découragée donc nous ne nous étendrons pas sur le sujet.

## Détails sur les objets *Store*

*Petit rappel*: Un *Store* traite une URI (dont la génération vient d'être décrite) pour agir "physiquement" sur une donnée (sur disque, sur une archive FTP, dans une base de données, ...)

Ce qui caractérise le mieux les *Store* est la notion "*d'interface*" qui existe dans des langages comme Java ou C++ (malheureusement cela n'existe pas nativement en Python). Pris sous l'angle de son *interface*, un *Store* serait un objet qui définit les méthodes suivantes :

* **locate**(*self*, *remote*, *options*=None)
* **check**(*self*, *remote*, *options*=None)
* **get**(*self*, *remote*, *local*, *options*=None)
* **put**(*self*, *local*, *remote*, *options*=None)
* **delete**(*self*, *remote*, *options*=None)

et les attributs (ou _properties_) suivants :

* **scheme**
* **netloc**
* **use_cache**
* **readonly**
* **writeable**


### Les stores "simples"

Il s'agit par exemple du store que nous avons utilisé dans notre exemple :

In [24]:
print(type(rh.store))
print(rh.store.use_cache())

<class 'vortex.data.stores.VortexOpArchiveStore'>
False


Ce *Store* répond *False* à **use_cache** ce qui signifie que l'accès aux données est relativement lent (en particulier, les fichiers ne peuvent pas être récupérés rapidement en lecture seule). En mode **put**, c'est typiquement le type de *Store* pour lesquels les données sont envoyées de façon asynchrone (via *ftserv* en R&D).

Par opposition, on trouve aussi des *Stores* rapides de type "cache". Concrètement, les *Store* de type "cache" sont :

* Hébergés sur un système de fichiers partagé rapide (Lustre) ou sur le disque dur si vous travaillez sur votre PC ;
* Idéalement, le cache doit se trouver sur le même système de fichiers que le répertoire de travail afin de pouvoir réaliser des liens durs ("hard links" de la norme POSIX).

Dans Vortex, il est fréquent de travailler avec des *Stores* de cache (pour ne pas récupérer plusieurs fois la même donnée sur la machine d'archivage, pour sauvegarder rapidement des résultats, ...).

Par exemple, il existe un *Store* de cache pour les données Vortex oper :

In [25]:
vsop_cache = fp.proxy.store(scheme='vortex', netloc='vsop.cache.fr')
print(type(vsop_cache))
print(vsop_cache.use_cache())

<class 'vortex.data.stores.VortexVsopCacheStore'>
True


D'accord, mais où sont stockées les données localement ?

Dans la plupart des cas, il n'y a pas à s'en soucier : Olive, le système de création de jobs officiel (présenté le dernier jour), ou le système de création de jobs oper, se chargent de configurer correctement les répertoires. 

Dans le cas où vous travaillez indépendamment (par exemple sur votre PC, pour récupérer telle ou telle donnée), il faut veiller à positioner la variable d'environnement ``MTOOLDIR`` (un sous-répertoire ``cache`` y sera créé par Vortex). Par défaut, le répertoire ``/tmp`` est utilisé (ce qui est rarement ce que l'on veut réellement faire).

### Le MultiStore

Le cas d'utilisation le plus courant est de recourir à un *Store* de cache ET un *Store* archive (au cas où la donnée ne serait pas/plus dans le cache). C'est exactement le rôle du *MultiStore* :

* Un *MultiStore* contient une liste (ordonnée) d'objets *Store* accessibles par son biais
* Un *MultiStore* satisfait en tout point "l'interface" *Store* décrite précédemment : son usage est donc transparent.
* Lors d'un **get**, le *MultiStore* essaye de satisfaire la demande avec le premier *Store* présent dans sa liste, puis le deuxième, ...
* Lors d'un **put** la ressource est déposée dans chacun des *Stores* (pour peu que ceux-ci ne soient pas *readonly*)

In [26]:
vsop_multi = fp.proxy.store(scheme='vortex', netloc='vsop.multi.fr')
print(type(vsop_multi))
# Liste des Stores sous-jacents...
vsop_multi.openedstores


<class 'vortex.data.stores.VortexStore'>


## Notion de namespace dans les *Providers*

Concrètement, ces notions de cache et de *MultiStore* sont importantes car elles se retrouvent dans l'utilisation courante de Vortex. Dans notre exemple, nous avions :

In [27]:
# Récupération de la ressource
rhandlers = toolbox.input(# Method/Section
                          role='InitialCondition', intent='inout',
                          # Resource
                          term=3, kind='gridpoint', origin='historic', nativefmt='grib',
                          # Provider
                          member=2, experiment='DBLE', namespace='vortex.archive.fr',
                          block='forecast',
                          # Container
                          local='GRIB_[geometry:area]_[member%04d]+[term:fmthm]', format='grib')
%ls
rhandlers[0].clear()

# [2019/01/24-14:03:25][vortex.data.stores][inarchiveget:1199][INFO]: inarchiveget on vortex://vsop.archive.fr//home/m/mxpt/mxpt001/vortex/arpege/pearp/DBLE/2017/06/01/T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0003:00.grib (to: GRIB_EUROC25_0002+0003:00)
# [2019/01/24-14:03:25][vortex.tools.storage][_ftpretrieve:0690][INFO]: ftpget on ftp://hendrix.meteo.fr//home/m/mxpt/mxpt001/vortex/arpege/pearp/DBLE/2017/06/01/T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0003:00.grib (to: GRIB_EUROC25_0002+0003:00)
# [2019/01/24-14:03:26][vortex.tools.net][get:0266][INFO]: FTP <get:/home/m/mxpt/mxpt001/vortex/arpege/pearp/DBLE/2017/06/01/T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0003:00.grib>


GRIB_EUROC25_0002+0003:00


True

Nous précisons explicitement ``vortex.archive.fr``, le resource's *Handler* va donc générer une URI qui conduit au recours inconditionel à l'archive (cela peut être voulu). Il est souvent plus intéressant de recourir à un *MultiStore* :

In [28]:
# Récupération de la ressource
rhandlers = toolbox.input(# Method/Section
                          role='InitialCondition', intent='inout',
                          # Resource
                          term=3, kind='gridpoint', origin='historic', nativefmt='grib',
                          # Provider
                          member=2, experiment='DBLE', namespace='vortex.multi.fr',
                          block='forecast',
                          # Container
                          local='GRIB_[geometry:area]_[member%04d]+[term:fmthm]', format='grib')
%ls
rhandlers[0].clear()

# [2019/01/24-14:03:26][vortex.data.stores][incacheget:1767][INFO]: incacheget on vortex://vsop.cache-mt.fr//arpege/pearp/DBLE/20170601T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0003:00.grib (to: GRIB_EUROC25_0002+0003:00)
# [2019/01/24-14:03:26][vortex.tools.systems][cp:2204][ERROR]: Missing source /home/meunierlf/cache/vortex/arpege/pearp/DBLE/20170601T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0003:00.grib
# [2019/01/24-14:03:26][vortex.data.stores][_verbose_log:0528][INFO]: Multistore get vortex://vsop.cache.fr: none of the opened store succeeded.
# [2019/01/24-14:03:26][vortex.data.stores][inarchiveget:1199][INFO]: inarchiveget on vortex://vsop.archive.fr//home/m/mxpt/mxpt001/vortex/arpege/pearp/DBLE/2017/06/01/T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0003:00.grib (to: GRIB_EUROC25_0002+0003:00)
# [2019/01/24-14:03:26][vortex.tools.storage][_ftpretrieve:0690][INFO]: ftpget on ftp://hendrix.meteo.fr//home/m/mxpt/mxpt001/vortex/arpege/pearp/DBLE/2017/06/01/T

GRIB_EUROC25_0002+0003:00


True

Dans la log, nous avons vu apparaître l'idée de "*refill*". Cette fonctionnalité, activée dans la plupart des *MultiStore*, implique que les fichiers récupérés en archive sont automatiquement redéposés dans les caches accessibles en écriture. Si le même fichier est récupéré ultérieurement, cela ira plus vite...

In [29]:
# Récupération de la ressource
rhandlers = toolbox.input(# Method/Section
                          role='InitialCondition', intent='inout',
                          # Resource
                          term=3, kind='gridpoint', origin='historic', nativefmt='grib',
                          # Provider
                          member=2, experiment='DBLE', namespace='vortex.multi.fr',
                          block='forecast',
                          # Container
                          local='GRIB_[geometry:area]_[member%04d]+[term:fmthm]', format='grib')
%ls
rhandlers[0].clear()

# [2019/01/24-14:03:27][vortex.data.stores][incacheget:1767][INFO]: incacheget on vortex://vsop.cache-mt.fr//arpege/pearp/DBLE/20170601T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0003:00.grib (to: GRIB_EUROC25_0002+0003:00)
# [2019/01/24-14:03:27][vortex.data.stores][incacheget:1781][INFO]: incacheget retrieve rc=True location=/home/meunierlf/cache/vortex/arpege/pearp/DBLE/20170601T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0003:00.grib


GRIB_EUROC25_0002+0003:00


True

In [30]:
# Nettoyage du cache pour que cela fonctionne lors du prochain lancement de la démo...
rhandlers[0].delete(incache=True)

# [2019/01/24-14:03:27][vortex.data.stores][incachedelete:1802][INFO]: incachedelete on vortex://vsop.cache-mt.fr//arpege/pearp/DBLE/20170601T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0003:00.grib
# [2019/01/24-14:03:27][vortex.data.stores][_verbose_log:0271][INFO]: Skip this store because a cache is requested


True

Note: Lorsqu'on utilise Olive ou l'outil de création de jobs "officiel" basé sur MTOOL, les caches sont automatiquement nettoyés au bout de quelques jours. Néanmoins, dans le cas général, le nettoyage est à la charge de l'utilisateur.

Cette notion de *namespace* est de l'ordre de la convention mais elle est assez respectée... Globalement, pour les *Provider* supportant cette notion de *namespace* :

* ``XXX.cache.fr`` : signifie que les données ne seront récupérées/déposées que dans le cache
* ``XXX.archive.fr`` : signifie que les données ne seront récupérées/déposées que dans l'archive
* ``XXX.multi.fr`` : signifie qu'on utilise à la fois un cache et l'archive.

La variante ``multi`` est en général ce que l'on veut lors de la récupération de données. Pour les données de sorties, la question se pose plus au cas par cas (on ne veut pas forcément archiver les fichiers intermédiaires).

Les providers suportant cette cette notion de namespace sont par exemple :

* le provider Vortex (que ce soit en R&D ou en oper) : ``vortex.[cache|archive|multi].fr``
* le provider pour l'archive DSI ksh : ``[oper|dble].[archive|multi].fr``
* le provider Olive : ``olive.[cache|archive|multi].fr``

Questions éventuelles : *vortex.support@meteo.fr*